In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
data0 = pd.read_csv("../input/covid19-osaka/osaka_summary - 0814.csv")
data0[-5:].T

In [ ]:
data0['% positive rate 7-day']=(data0['examined positives']*100/data0['examined']).rolling(window=7).mean()
data0['positives mean 7-day']=data0['examined positives'].rolling(window=7).mean()
data0['positives max 7-day']=data0['examined positives'].rolling(window=7).max()
data0['positives min 7-day']=data0['examined positives'].rolling(window=7).min()
data0[-5:].T

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data0['date'],y=data0['positives mean 7-day'],name='positives mean 7-day'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data0['date'],y=data0['positives max 7-day'],name='positives max 7-day'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data0['date'],y=data0['positives min 7-day'],name='positives min 7-day'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Examined Positives (rolling 7-day) in Osaka")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Cases",secondary_y=False)
fig.show()

In [ ]:
col0=data0.columns.to_list()
col1=col0+['pm-7','slope']
data1=pd.DataFrame(columns=col1)
data1[col0]=data0

In [ ]:
n=len(data0)
for  i in range(n):    
    pmi=data1['positives mean 7-day'][i]
    data1.loc[i+7,'pm-7']=pmi

In [ ]:
# Slope value and Rt (simplified effective reproduction number) were defined as follows.
# 'pm-7' means 'positives mean 7-day' value 7 days before.

data1['slope']=(data1['positives mean 7-day']-data1['pm-7'])/7
data1['Rt']=data1['positives mean 7-day']/data1['pm-7']  
data2=data1[['date','examined positives','positives mean 7-day','slope','Rt']]
data3=data2[14:-7]

print(data3[250:265]) ### 3rd wave
print()
print(data3[395:410]) ### 4th wave
print()
print(data3[508:523]) ### 5th wave
print()
print(data3[-14:]) ### latest

# A slope value is a sensitive benchmark for an occurence of infectious explosion in the near future.


* On 2020-10-24, the slope value exceeded 3.0. This was the begginning of the 3th wave.
* On 2021-03-17, the slope value exceeded 3.0. This was the begginning of the 4th wave.
* On 2021-07-07, the slope value exceeded 3.0. This is the begginning of the 5th wave.
* Cf. The situation in Tokyo: https://www.kaggle.com/stpeteishii/covid19-tokyo-infection-situation2

# The peak of the 5th wave has gone? 
* The slope has reached more than 10 since 2021/07/15.
* The slope has reached more than 20 since 2021/07/27.
* The slope has reached more than 40 since 2021/07/30.

In [ ]:
print('The peak of 4th wave') ### 5 days
print(data3[427:434]) ### more than 40, MAX 43

In [ ]:
print('The peak of 5th wave') ### 8 days
print(data3[536:546]) ### more than 40, MAX 62

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data3['date'][300:],y=data3['Rt'][300:],name='Rt'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Rt (simplified effective reproduction number) change in Osaka")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=True)
fig.show()

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data3['date'][300:],y=data3['slope'][300:],name='slope'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Slope change of 7-day mean in Osaka")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=True)
fig.show()

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":True}]])
fig.add_trace(go.Scatter(x=data3['date'],y=data3['slope'],name='slope'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['date'],y=data3['positives mean 7-day'],name='positives mean 7-day'),secondary_y='positives mean 7-day')
fig.update_layout(autosize=False,width=700,height=500,title_text="Slope change of 7-day mean in Osaka")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Slope",secondary_y=False)
fig.update_yaxes(title_text="positives mean 7-day",secondary_y=True)
fig.show()